In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sc

This code is made to clean all of the data collected by the WebCrawler notebook. 

In [2]:
"""
Array 0: Name of all congress people who vote on a bill
Array 1: State the congress people are from
Array 2: party of congress people
Array 3: How they voted on a give bill

!!! This is a ragged array because not all congress people vote on every bill !!!
"""

#Read in all lines from file
f = open("data.csv", "r")
array = []
line = f.readline()
index = 0
while line:
    line = line.strip("\n")
    line = line.split()
    array.append([])
    for item in line:
        array[index].append(item)
    line = f.readline()
    index += 1
f.close()

In [3]:
#Reorganize data
allData = []
for j in range(0,len(array)):
    allArray = []
    data = []
    for i in range(0,len(array[j])):
        if array[j][i][0] == '[':
            data.append(array[j][i][1:])
        elif array[j][i][-1] == ']':
            data.append(array[j][i][:-1])
            allArray.append(data)
            data = []
        else:
            data.append(array[j][i])
        
    allData.append(allArray)


In [5]:
#clean state array
#Any 2 word state is a problem, need to concatenate the words since they are saved as different elements

#print(np.unique(allData[1][0]))
problems = ["'New","'West", "'North","'South","'Rhode"]

for j in range(0,len(allData[1])):
    bad_index = []
    for i in range(0,len(allData[1][j])):
        if allData[1][j][i] in problems:
            allData[1][j][i] = allData[1][j][i] +"_"+ allData[1][j][i+1]
            bad_index.append(i+1)

    allData[1][j] = np.delete(allData[1][j], bad_index)
    allData[1][j] = np.delete(allData[1][j], np.argwhere(allData[1][j] == 'None,'))
    


In [6]:
#Clean party array
for j in range(0,2):
    for i in range(0,len(allData[2])):
        allData[2][i] = np.delete(allData[2][i], np.argwhere(allData[2][i] == 'None,'))
        

In [7]:
#clean vote array
for j in range(0,len(allData[3])):
    bad_index = []
    for i in range(0,len(allData[3][j])):
        if allData[3][j][i] == "'Not":
            allData[3][j][i] = allData[3][j][i] +"_"+ allData[3][j][i+1]
            bad_index.append(i+1)
        elif allData[3][j][i] == "'Ye'":
            allData[3][j][i] = 'Yea'

    allData[3][j] = np.delete(allData[3][j], bad_index)
    allData[3][j] = np.delete(allData[3][j], np.argwhere(allData[3][j] == 'None,'))



In [8]:
#Remove extra "" from every element
for k in range(0,len(allData)):
    for j in range(0,len(allData[k])):
        for i in range(0,len(allData[k][j])):
            allData[k][j][i] = allData[k][j][i][1:-2]
            if allData[k][j][i] == 'Republica':
                 allData[k][j][i] = 'Republican'


In [9]:
names = allData[0][0]

for i in range(1,len(allData[0])):
    
    diff = np.setdiff1d(allData[0][i], names)
    if len(diff) > 0:
        for j in diff:
            names.append(j)
names = np.sort(names)

In [10]:
states = np.empty(len(names),dtype = object)
party = np.empty(len(names),dtype = object)
for j in range(0,len(allData[1])):
    for k in range(0,len(allData[1][j])):
        for i in range(0,len(names)):
            if names[i] == allData[0][j][k]:
                states[i] = allData[1][j][k]
                party[i] = allData[2][j][k]
               
            
    

In [11]:
#for j in range(0,len(allData[3])):
allVotes = []
for j in range(0,len(allData[3])):
    votes = np.full(len(names), 'Not_Voting')
    for i in range(0,len(allData[3][j])):
        votes[np.where(names == allData[0][j][i])] = allData[3][j][i]
    allVotes.append(votes)
        

In [12]:
stuff = [names,states,party]
labels = ["Name","State", "Party"]
count = 1;
for i in allVotes:
    stuff.append(i)
    labels.append("Bill_"+str(count))
    count+=1
    

In [13]:
df = pd.DataFrame(np.array(stuff).T,columns =labels)
df

,Name,State,Party,Bill_1,Bill_2,Bill_3,Bill_4,Bill_5,Bill_6,Bill_7,...,Bill_301,Bill_302,Bill_303,Bill_304,Bill_305,Bill_306,Bill_307,Bill_308,Bill_309,Bill_310
0,a_dutch_ruppersberger,Maryland,Democrat,Not_Voting,Yea,Yea,Yea,Nay,Yea,Yea,...,Yea,Yea,Yea,Yea,Yea,Yea,Nay,Yea,Yea,Yea
1,abigail_spanberger,Virginia,Democrat,Not_Voting,Yea,Yea,Yea,Nay,Yea,Yea,...,Yea,Yea,Yea,Yea,Yea,Yea,Nay,Yea,Yea,Yea
2,adam_kinzinger,Illinois,Republican,Yea,Nay,Nay,Nay,Yea,Nay,Yea,...,Yea,Not_Voting,Nay,Nay,Yea,Yea,Yea,Yea,Yea,Yea
3,adam_schiff,California,Democrat,Not_Voting,Yea,Yea,Yea,Nay,Yea,Yea,...,Yea,Yea,Yea,Yea,Yea,Yea,Nay,Yea,Yea,Yea
4,adam_smith,Washington,Democrat,Yea,Yea,Yea,Yea,Nay,Yea,Yea,...,Yea,Yea,Yea,Not_Voting,Yea,Yea,Nay,Yea,Yea,Yea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,william_timmons,South_Carolina,Republican,Yea,Nay,Nay,Nay,Yea,Nay,Yea,...,Yea,Yea,Nay,Nay,Yea,Nay,Yea,Nay,Nay,Yea
434,young_kim,California,Republican,Yea,Nay,Nay,Nay,Yea,Nay,Yea,...,Yea,Yea,Nay,Nay,Yea,Yea,Yea,Nay,Yea,Yea
435,yvette_clarke,New_York,Democrat,Yea,Yea,Yea,Yea,Nay,Yea,Yea,...,Yea,Yea,Yea,Yea,Yea,Yea,Nay,Yea,Yea,Yea
436,yvette_herrell,New_Mexico,Republican,Yea,Nay,Nay,Nay,Yea,Nay,Yea,...,Yea,Yea,Nay,Nay,Yea,Nay,Yea,Nay,Nay,Yea


There were some random sparse errors still in the data, the code below fixes theses minor errors

In [15]:
for i in range(1,311):
    for j in range(0,len(df["Bill_" + str(i)])):
        if df["Bill_" + str(i)][j] == "Ye":
            df["Bill_" + str(i)][j] = "Yea"

In [16]:
for i in range(1,311):
    if np.sum(np.where(df["Bill_" + str(i)]=='Na')) != 0:
        df["Bill_" + str(i)][265] = "Nay"
       

In [17]:
for i in range(1,311):
    for j in range(0,len(df["Bill_" + str(i)])):
        if df["Bill_" + str(i)][j] == "Not_Votin" or df["Bill_" + str(i)][j] == "Present":
            df["Bill_" + str(i)][j] = "Not_Voting"

The code below is data interpolation, every not voting is converted to a vote for their party on that bill

In [18]:
repMode = []
demMode = []

for i in range(1,311):

    mode1 = sc.mode(df[df["Party"]=="Republican"]["Bill_"+str(i)])[0][0]
    repMode.append(mode1)
    
    mode2 = sc.mode(df[df["Party"]=="Democrat"]["Bill_"+str(i)])[0][0]
    demMode.append(mode2)

In [19]:
for i in range(1,311):
    for j in range(0,len(df["Bill_" + str(i)])):
        if df["Bill_" + str(i)][j] == "Not_Voting":
            if df['Party'][j] == "Republican":
                df["Bill_" + str(i)][j] = repMode[i-1]
            else:
                 df["Bill_" + str(i)][j] = demMode[i-1]
                
        

In [22]:
#save data to csv
df.to_csv("cleanData.csv")